<a href="https://colab.research.google.com/github/spps-supalerk/CRM_Analytics/blob/main/CRM_Analytics_Customer_Behaviors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

# For download file from file_id which are stored in GGdrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Clean Data

## Load File from My Google Drive

In [2]:
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1mkGRWGAFzixktIdVACMEh_qzGmwHNe7o'
f_dir = 'Customer_Behaviors.csv'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(f_dir)

In [3]:
df_cb = pd.read_csv(f_dir)
df_cb.head()

,Timestamp,คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารญี่ปุ่น],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารจึน],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารไทย],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารอีสาน],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารสุขภาพ],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [บุฟเฟต์],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [ชาบูชาบู],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [ปิ้งย่าง],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [เหล้าเบียร์],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [เสื้อผ้าแฟชั่น],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [แต่งหน้า],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [skincare บำรุงผิว],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [การออกกำลังกาย],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [การเล่นเกมส์],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [การท่องเที่ยว],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [การอ่านหนังสือ],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [การอ่านบทความ Online],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [การเรียน e-learning],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [ดู Series],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [ดูหนัง],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [ดู Youtube],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [ดู Netflix],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [Content ด้านบันเทิง],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [Content เกี่่ยวกับเกมส์],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [Content เกี่ยวกับการ review ร้านอาหาร],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [Content เกี่ยวกับการท่องเที่ยว],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [Content เกี่ยวกับการแต่งบ้าน],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [Content เกี่ยวกับการโทรทัศน์],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [Content เกี่ยวกับละคร],คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [Content เกี่ยวกับการเมือง],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [อาหารญี่ปุ่น],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [อาหารจึน],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [อาหารไทย],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [อาหารอีสาน],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [อาหารสุขภาพ],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [บุฟเฟต์],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [ชาบูชาบู],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [ปิ้งย่าง],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [เหล้าเบียร์],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [เสื้อผ้าแฟชั่น],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [แต่งหน้า],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [skincare บำรุงผิว],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [การออกกำลังกาย],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [การเล่นเกมส์],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [การท่องเที่ยว],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [การอ่านหนังสือ],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [การอ่านบทความ Online],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [การเรียน e-learning],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [ดู Series],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [ดูหนัง],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [ดู Youtube],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [ดู Netflix],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content ด้านบันเทิง],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่่ยวกับเกมส์],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการ review ร้านอาหาร],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการท่องเที่ยว],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการแต่งบ้าน],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการโทรทัศน์],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับละคร],คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการเมือง],นอกเหนือจากรายการดังกล่าวแล้ว คุณมีความสนใจในเรื่องใดอีกบ้าง,คุณเกิดวันที่เท่าไหร่,เพศ
0,1-9-2021 10:07:27,สนใจ,สนใจ,สนใจอย่างมากที่สุด,สนใจอย่างมาก,สนใจ,สนใจ,สนใจ,สนใจ,สนใจอย่างมากที่สุด,เฉยๆ,ไม่สนใจ,สนใจ,สนใจอย่างมาก,สนใจอย่างมาก,เฉยๆ,สนใจ,สนใจ,สนใจอย่างมากที่สุด,สนใจอย่างมาก,สนใจอย่างมาก,สนใจ,สนใจอย่างมากที่สุด,สนใจ,สนใจอย่างมาก,สนใจอย่างมาก,เฉยๆ,เฉยๆ,เฉยๆ,เฉยๆ,สนใจอย่างมากที่สุด,เดือนละ 2-3 ครั้ง,หลายเดือนครั้ง,แทบทุกวัน,อาทิตย์ละครั้ง,แทบทุกวัน,เดือนละ 2-3 ครั้ง,เดือนละ 2-3 ครั้ง,เดือนละครั้ง,อาทิตย์ละครั้ง,หลายเดือนครั้ง,หลายเดือนครั้ง,แทบทุกวัน,แทบทุกวัน,อาทิตย์ละครั้ง,เดือนละ 2-3 ครั้ง,เดือนละ 2-3 ครั้ง,เดือนละ 2-3 ครั้ง,เด

In [4]:
print(df_cb.columns[:10])
print(df_cb.columns[-10:])

Index(['Timestamp', 'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารญี่ปุ่น]',
       'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารจึน]',
       'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารไทย]',
       'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารอีสาน]',
       'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [อาหารสุขภาพ]',
       'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [บุฟเฟต์]',
       'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [ชาบูชาบู]',
       'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [ปิ้งย่าง]',
       'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด [เหล้าเบียร์]'],
      dtype='object')
Index(['คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่่ยวกับเกมส์]',
       'คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการ review ร้านอาหาร]',
       'คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการท่องเที่ยว]',
       'คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการแต่งบ้าน]',
       'คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน [Content เกี่ยวกับการโทรทัศน์]',
       'คุ

## Clean Columns Name

In [5]:
new_col = []
for c in df_cb.columns:
  c = re.sub("[\[\]]+", '',c)
  if c == 'นอกเหนือจากรายการดังกล่าวแล้ว คุณมีความสนใจในเรื่องใดอีกบ้าง':
    c = 'ความสนใจอื่นๆ'
  elif 'คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด' in c:
    c = c.replace('คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด', 'สนใจ')
  elif 'คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน' in c:
    c = c.replace('คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน', 'บริโภค')
  else:
    print('skip :', c)
  c = c.replace(' ','_')
  new_col.append(c)
df_cb.columns = new_col
print('Example of column names')
print(df_cb.columns[:10])
print(df_cb.columns[-10:])

# categirize column groups
col_interest = [ c for c in df_cb.columns if 'สนใจ' in c and c != 'นอกเหนือจากรายการดังกล่าวแล้ว_คุณมีความสนใจในเรื่องใดอีกบ้าง']
col_consume = [ c for c in df_cb.columns if 'บริโภค' in c]
col_free_text = ['ความสนใจอื่นๆ']

print(f'col_interest has total {len(col_interest)} cols')
print(col_interest,'\n')
print(f'col_consume has total {len(col_consume)} cols')
print(col_consume,'\n')
print(f'col_free_text has total {len(col_free_text)} col')
print(col_free_text)

skip : Timestamp
skip : คุณเกิดวันที่เท่าไหร่
skip : เพศ
Example of column names
Index(['Timestamp', 'สนใจ_อาหารญี่ปุ่น', 'สนใจ_อาหารจึน', 'สนใจ_อาหารไทย',
       'สนใจ_อาหารอีสาน', 'สนใจ_อาหารสุขภาพ', 'สนใจ_บุฟเฟต์', 'สนใจ_ชาบูชาบู',
       'สนใจ_ปิ้งย่าง', 'สนใจ_เหล้าเบียร์'],
      dtype='object')
Index(['บริโภค_Content_เกี่่ยวกับเกมส์',
       'บริโภค_Content_เกี่ยวกับการ_review_ร้านอาหาร',
       'บริโภค_Content_เกี่ยวกับการท่องเที่ยว',
       'บริโภค_Content_เกี่ยวกับการแต่งบ้าน',
       'บริโภค_Content_เกี่ยวกับการโทรทัศน์', 'บริโภค_Content_เกี่ยวกับละคร',
       'บริโภค_Content_เกี่ยวกับการเมือง', 'ความสนใจอื่นๆ',
       'คุณเกิดวันที่เท่าไหร่', 'เพศ'],
      dtype='object')
col_interest has total 31 cols
['สนใจ_อาหารญี่ปุ่น', 'สนใจ_อาหารจึน', 'สนใจ_อาหารไทย', 'สนใจ_อาหารอีสาน', 'สนใจ_อาหารสุขภาพ', 'สนใจ_บุฟเฟต์', 'สนใจ_ชาบูชาบู', 'สนใจ_ปิ้งย่าง', 'สนใจ_เหล้าเบียร์', 'สนใจ_เสื้อผ้าแฟชั่น', 'สนใจ_แต่งหน้า', 'สนใจ_skincare_บำรุงผิว', 'สนใจ_การออกกำลังกาย', 'สนใจ_การเล่นเกมส์', 'ส

## Likert Scaler

In [6]:
replace_likert_interest = {
    'ไม่สนใจอย่างมากที่สุด' : 0,
    'ไม่สนใจอย่างมาก' : 1,
    'ไม่สนใจ' : 2,
    'เฉยๆ' : 3,
    'สนใจ' : 4,
    'สนใจอย่างมาก' : 5,
    'สนใจอย่างมากที่สุด' : 6
}

replace_likert_consume = {
    'แทบไม่ได้บริโภคเลย': 0,
    'หลายเดือนครั้ง' : 1,
    'เดือนละ 2-3 ครั้ง' : 2,
    'เดือนละครั้ง' : 3,
    'อาทิตย์ละครั้ง' : 4,
    'แทบทุกวัน' : 5
}

df_cb[col_interest] = df_cb[col_interest].replace(replace_likert_interest)
df_cb[col_consume] = df_cb[col_consume].replace(replace_likert_consume)
df_cb.head(10)

,Timestamp,สนใจ_อาหารญี่ปุ่น,สนใจ_อาหารจึน,สนใจ_อาหารไทย,สนใจ_อาหารอีสาน,สนใจ_อาหารสุขภาพ,สนใจ_บุฟเฟต์,สนใจ_ชาบูชาบู,สนใจ_ปิ้งย่าง,สนใจ_เหล้าเบียร์,สนใจ_เสื้อผ้าแฟชั่น,สนใจ_แต่งหน้า,สนใจ_skincare_บำรุงผิว,สนใจ_การออกกำลังกาย,สนใจ_การเล่นเกมส์,สนใจ_การท่องเที่ยว,สนใจ_การอ่านหนังสือ,สนใจ_การอ่านบทความ_Online,สนใจ_การเรียน_e-learning,สนใจ_ดู_Series,สนใจ_ดูหนัง,สนใจ_ดู_Youtube,สนใจ_ดู_Netflix,สนใจ_Content_ด้านบันเทิง,สนใจ_Content_เกี่่ยวกับเกมส์,สนใจ_Content_เกี่ยวกับการ_review_ร้านอาหาร,สนใจ_Content_เกี่ยวกับการท่องเที่ยว,สนใจ_Content_เกี่ยวกับการแต่งบ้าน,สนใจ_Content_เกี่ยวกับการโทรทัศน์,สนใจ_Content_เกี่ยวกับละคร,สนใจ_Content_เกี่ยวกับการเมือง,บริโภค_อาหารญี่ปุ่น,บริโภค_อาหารจึน,บริโภค_อาหารไทย,บริโภค_อาหารอีสาน,บริโภค_อาหารสุขภาพ,บริโภค_บุฟเฟต์,บริโภค_ชาบูชาบู,บริโภค_ปิ้งย่าง,บริโภค_เหล้าเบียร์,บริโภค_เสื้อผ้าแฟชั่น,บริโภค_แต่งหน้า,บริโภค_skincare_บำรุงผิว,บริโภค_การออกกำลังกาย,บริโภค_การเล่นเกมส์,บริโภค_การท่องเที่ยว,บริโภค_การอ่านหนังสือ,บริโภค_การอ่านบทความ_Online,บริโภค_การเรียน_e-learning,บริโภค_ดู_Series,บริโภค_ดูหนัง,บริโภค_ดู_Youtube,บริโภค_ดู_Netflix,บริโภค_Content_ด้านบันเทิง,บริโภค_Content_เกี่่ยวกับเกมส์,บริโภค_Content_เกี่ยวกับการ_review_ร้านอาหาร,บริโภค_Content_เกี่ยวกับการท่องเที่ยว,บริโภค_Content_เกี่ยวกับการแต่งบ้าน,บริโภค_Content_เกี่ยวกับการโทรทัศน์,บริโภค_Content_เกี่ยวกับละคร,บริโภค_Content_เกี่ยวกับการเมือง,ความสนใจอื่นๆ,คุณเกิดวันที่เท่าไหร่,เพศ
0,1-9-2021 10:07:27,4,4,6,5,4,4,4,4,6,3,2,4,5,5,3,4,4,6,5,5,4,6,4,5,5,3,3,3,3,6,2,1,5,4,5,2,2,3,4,1,1,5,5,4,2,2,2,2,2,4,4,4,1,3,2,2,2,2,2,5,"กีฬา UFC, Soccer",19-09-88,ชาย
1,1-9-2021 10:07:34,5,3,6,5,4,5,6,5,2,3,3,3,5,5,6,6,6,6,3,6,6,5,3,3,5,6,6,3,2,4,4,1,5,4,4,2,2,2,0,3,0,3,5,5,3,5,5,4,1,4,5,2,3,2,5,5,4,1,1,4,กีฬา สัตว์เลี้ยง เทคโนโลยี,10-08-36,หญิง
2,1-9-2021 10:07:36,6,3,5,4,3,6,6,6,1,3,0,1,4,6,6,6,6,3,6,6,6,6,6,6,6,6,6,5,2,6,4,1,5,1,1,3,3,3,0,0,0,0,1,4,1,5,2,2,5,5,5,5,5,5,5,2,2,2,1,5,-,07-11-93,ชาย
3,1-9-2021 10:07:36,4,3,3,3,4,4,4,4,0,0,0,6,5,6,6,4,4,5,6,6,6,6,0,6,3,4,3,2,0,5,3,0,5,2,0,3,3,3,0,0,0,5,3,2,1,2,2,2,4,4,4,4,0,5,3,3,0,0,0,2,"โหราศาสตร์,ดูดวง",14-04-94,ชาย
4,1-9-2021 10:07:42,5,4,4,4,6,3,3,3,3,3,4,5,6,2,4,5,4,4,6,6,3,6,3,2,3,4,4,2,2,3,2,3,4,4,5,1,1,1,3,3,5,5,5,0,1,2,4,2,4,4,2,4,3,0,3,2,3,0,0,2,สัตว์เลี้ยง,19-04-95,หญิง
5,1-9-2021 10:07:42,6,6,6,4,3,6,6,6,6,2,2,2,3,4,6,0,0,6,6,6,6,6,6,6,6,6,6,6,6,6,3,3,5,2,0,3,3,3,1,0,0,0,0,1,1,0,3,5,2,2,5,2,5,1,3,3,3,3,3,5,ด้านการออมเงินวางแผนการใช้เงิน,25-04-93,ชาย
6,1-9-2021 10:07:53,6,4,6,6,4,3,4,4,3,6,6,6,4,2,6,5,5,5,5,5,5,5,5,1,5,6,4,4,5,5,4,3,5,5,2,3,3,3,0,2,5,5,2,0,3,4,4,4,4,4,4,4,5,0,2,2,2,5,5,5,อุปกรณ์ไอที,17-05-92,หญิง
7,1-9-2021 10:08:05,5,2,6,6,4,5,6,4,6,4,1,2,4,3,6,5,5,4,5,5,4,5,3,2,3,5,5,3,2,5,4,0,5,4,2,4,3,1,4,3,0,1,2,3,3,2,5,1,3,3,5,4,1,1,4,5,4,0,0,5,"ฟุตบอล, ถ่ายภาพ, รถยนต์, ผู้หญิง",02-04-93,ชาย
8,1-9-2021 10:08:08,6,2,5,5,3,5,5,5,2,3,0,4,4,6,5,5,5,5,6,6,6,6,6,6,6,5,3,3,3,5,4,0,5,4,1,2,2,2,0,3,0,5,3,5,3,2,5,4,2,2,5,2,4,4,4,3,3,3,3,4,"Technology, Gadget, Shopping, Business",29-12-93,ชาย
9,1-9-2021 10:08:10,4,3,5,3,4,4,4,4,5,4,4,4,5,4,4,4,4,5,4,5,5,4,4,4,4,4,4,4,4,5,5,1,5,2,4,3,3,3,2,2,1,5,5,5,3,4,2,5,4,4,4,4,4,2,2,2,2,2,2,5,นอน,11-11-85,ชาย


## Age Calculation

In [7]:
df_cb['คุณเกิดวันที่เท่าไหร่'] = pd.to_datetime(df_cb['คุณเกิดวันที่เท่าไหร่'])
df_cb['birth_year'] = df_cb['คุณเกิดวันที่เท่าไหร่'].dt.year
# if birth_year >= 2020, change BC by 2564(BE) - x + 500 ; 2064 - x
df_cb['age'] = df_cb['birth_year'].apply(lambda x: 2021 - x if x <= 2020 else 2064 - x )
df_cb.head()

,Timestamp,สนใจ_อาหารญี่ปุ่น,สนใจ_อาหารจึน,สนใจ_อาหารไทย,สนใจ_อาหารอีสาน,สนใจ_อาหารสุขภาพ,สนใจ_บุฟเฟต์,สนใจ_ชาบูชาบู,สนใจ_ปิ้งย่าง,สนใจ_เหล้าเบียร์,สนใจ_เสื้อผ้าแฟชั่น,สนใจ_แต่งหน้า,สนใจ_skincare_บำรุงผิว,สนใจ_การออกกำลังกาย,สนใจ_การเล่นเกมส์,สนใจ_การท่องเที่ยว,สนใจ_การอ่านหนังสือ,สนใจ_การอ่านบทความ_Online,สนใจ_การเรียน_e-learning,สนใจ_ดู_Series,สนใจ_ดูหนัง,สนใจ_ดู_Youtube,สนใจ_ดู_Netflix,สนใจ_Content_ด้านบันเทิง,สนใจ_Content_เกี่่ยวกับเกมส์,สนใจ_Content_เกี่ยวกับการ_review_ร้านอาหาร,สนใจ_Content_เกี่ยวกับการท่องเที่ยว,สนใจ_Content_เกี่ยวกับการแต่งบ้าน,สนใจ_Content_เกี่ยวกับการโทรทัศน์,สนใจ_Content_เกี่ยวกับละคร,สนใจ_Content_เกี่ยวกับการเมือง,บริโภค_อาหารญี่ปุ่น,บริโภค_อาหารจึน,บริโภค_อาหารไทย,บริโภค_อาหารอีสาน,บริโภค_อาหารสุขภาพ,บริโภค_บุฟเฟต์,บริโภค_ชาบูชาบู,บริโภค_ปิ้งย่าง,บริโภค_เหล้าเบียร์,บริโภค_เสื้อผ้าแฟชั่น,บริโภค_แต่งหน้า,บริโภค_skincare_บำรุงผิว,บริโภค_การออกกำลังกาย,บริโภค_การเล่นเกมส์,บริโภค_การท่องเที่ยว,บริโภค_การอ่านหนังสือ,บริโภค_การอ่านบทความ_Online,บริโภค_การเรียน_e-learning,บริโภค_ดู_Series,บริโภค_ดูหนัง,บริโภค_ดู_Youtube,บริโภค_ดู_Netflix,บริโภค_Content_ด้านบันเทิง,บริโภค_Content_เกี่่ยวกับเกมส์,บริโภค_Content_เกี่ยวกับการ_review_ร้านอาหาร,บริโภค_Content_เกี่ยวกับการท่องเที่ยว,บริโภค_Content_เกี่ยวกับการแต่งบ้าน,บริโภค_Content_เกี่ยวกับการโทรทัศน์,บริโภค_Content_เกี่ยวกับละคร,บริโภค_Content_เกี่ยวกับการเมือง,ความสนใจอื่นๆ,คุณเกิดวันที่เท่าไหร่,เพศ,birth_year,age
0,1-9-2021 10:07:27,4,4,6,5,4,4,4,4,6,3,2,4,5,5,3,4,4,6,5,5,4,6,4,5,5,3,3,3,3,6,2,1,5,4,5,2,2,3,4,1,1,5,5,4,2,2,2,2,2,4,4,4,1,3,2,2,2,2,2,5,"กีฬา UFC, Soccer",1988-09-19,ชาย,1988,33
1,1-9-2021 10:07:34,5,3,6,5,4,5,6,5,2,3,3,3,5,5,6,6,6,6,3,6,6,5,3,3,5,6,6,3,2,4,4,1,5,4,4,2,2,2,0,3,0,3,5,5,3,5,5,4,1,4,5,2,3,2,5,5,4,1,1,4,กีฬา สัตว์เลี้ยง เทคโนโลยี,2036-10-08,หญิง,2036,28
2,1-9-2021 10:07:36,6,3,5,4,3,6,6,6,1,3,0,1,4,6,6,6,6,3,6,6,6,6,6,6,6,6,6,5,2,6,4,1,5,1,1,3,3,3,0,0,0,0,1,4,1,5,2,2,5,5,5,5,5,5,5,2,2,2,1,5,-,1993-07-11,ชาย,1993,28
3,1-9-2021 10:07:36,4,3,3,3,4,4,4,4,0,0,0,6,5,6,6,4,4,5,6,6,6,6,0,6,3,4,3,2,0,5,3,0,5,2,0,3,3,3,0,0,0,5,3,2,1,2,2,2,4,4,4,4,0,5,3,3,0,0,0,2,"โหราศาสตร์,ดูดวง",1994-04-14,ชาย,1994,27
4,1-9-2021 10:07:42,5,4,4,4,6,3,3,3,3,3,4,5,6,2,4,5,4,4,6,6,3,6,3,2,3,4,4,2,2,3,2,3,4,4,5,1,1,1,3,3,5,5,5,0,1,2,4,2,4,4,2,4,3,0,3,2,3,0,0,2,สัตว์เลี้ยง,1995-04-19,หญิง,1995,26


In [8]:
df_cb.reset_index().to_csv('questionnaires.csv')

# Pearson Correlation

In [9]:
df_cb_corr = df_cb.corr().reset_index().melt(id_vars = 'index')
df_cb_corr.columns = ['var1', 'var2' , 'pearson_corr']
df_cb_corr.to_csv('cb_corr.csv',index=False)

# Free Text Analysis Using Universal Sentence Encoder for Categorizing

In [10]:
def split_text(x):
  x = x.lower()
  x = x.replace('และ','')
  x = x.replace(' and ','')
  if ',' in x:
    x = x.replace(' ','')
    lst_text = x.split(',')
  else:
    lst_text = x.split(' ')
  return lst_text

In [11]:
df_cb['ความสนใจอื่นๆ'].head(50)

0                                      กีฬา UFC, Soccer
1                            กีฬา สัตว์เลี้ยง เทคโนโลยี
2                                                     -
3                                      โหราศาสตร์,ดูดวง
4                                           สัตว์เลี้ยง
5                        ด้านการออมเงินวางแผนการใช้เงิน
6                                           อุปกรณ์ไอที
7                      ฟุตบอล, ถ่ายภาพ, รถยนต์, ผู้หญิง
8                Technology, Gadget, Shopping, Business
9                                                   นอน
10                                          สัตว์เลี้ยง
11                                           ปลูกต้นไม้
12                                       วงการ YouTuber
13                                           การออมเงิน
14                                                ดนตรี
15                          สัตว์เลี้ยง น้องหมา น้องแมว
16                                        ขายของออนไลน์
17                                              

### Download USE Pre-Trained Model 

In [12]:
%tensorflow_version 2.x

In [13]:
!pip install tensorflow_text

In [14]:
import tensorflow_hub as hub
import tensorflow_text
import tensorflow as tf
import json
import itertools

In [15]:
#download open subtitle EN-TH dataset
!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/moses/en-th.txt.zip

--2021-01-15 07:53:24--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/moses/en-th.txt.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/moses/en-th.txt.zip [following]
--2021-01-15 07:53:26--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/moses/en-th.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119520928 (114M) [application/zip]
Saving to: ‘download.php?f=OpenSubtitles%2Fv2018%2Fmoses%2Fen-th.txt.zip.1’

download.php?f=Open 100%[===================>] 113.98M  17.0MB/s    in 7.9s    

2021-01-15 07:53:35 (14.4 MB/s) - ‘download.php?f=OpenSubtitles%2Fv2018%2Fmoses%2Fen-th.txt.zip.1’ saved [119520

In [16]:
!unzip download.php?f=OpenSubtitles%2Fv2018%2Fmoses%2Fen-th.txt.zip -d data

Archive:  download.php?f=OpenSubtitles%2Fv2018%2Fmoses%2Fen-th.txt.zip
replace data/OpenSubtitles.en-th.en? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/OpenSubtitles.en-th.en  
replace data/OpenSubtitles.en-th.th? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/OpenSubtitles.en-th.th  
replace data/OpenSubtitles.en-th.ids? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/OpenSubtitles.en-th.ids  
replace data/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/README             


In [17]:
#Create USE embedding by using tensorflow hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

### Applied USE to Other Interest Items

In [18]:
other_interest = df_cb['ความสนใจอื่นๆ'].apply(split_text).explode()
other_interest = other_interest[other_interest.apply(len)>=2]
lst_ignore = ['content','(ไม่แน่ใจว่าจริงไหม','เช่น']
other_interest = other_interest[~other_interest.isin(lst_ignore)].reset_index()
other_interest.head(10)

,index,ความสนใจอื่นๆ
0,0,กีฬาufc
1,0,soccer
2,1,กีฬา
3,1,สัตว์เลี้ยง
4,1,เทคโนโลยี
5,3,โหราศาสตร์
6,3,ดูดวง
7,4,สัตว์เลี้ยง
8,5,ด้านการออมเงินวางแผนการใช้เงิน
9,6,อุปกรณ์ไอที


In [19]:
df_use_oth_interest = pd.DataFrame(embed(other_interest['ความสนใจอื่นๆ'].values).numpy(), index = other_interest['ความสนใจอื่นๆ'].values)
df_use_oth_interest.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
กีฬาufc,0.106929,-0.040677,0.039707,-0.050232,-0.011018,0.026707,0.061621,-0.032751,-0.037130,-0.019983,0.041342,-0.022466,-0.072359,0.008045,0.038108,0.020356,-0.036452,0.034335,0.041557,0.059720,0.028891,0.069184,0.029928,0.002577,-0.022055,0.041563,0.013456,-0.019306,-0.015649,-0.032495,0.005222,0.010471,-0.013591,-0.033576,0.063754,0.003958,0.048139,0.081499,-0.007863,0.076855,...,0.107795,0.053800,0.054311,0.052866,-0.064976,-0.003409,0.014319,0.015612,-0.039938,0.037555,0.016003,0.009080,-0.010312,-0.032720,0.006448,-0.069425,0.049810,-0.002707,0.004867,-0.094720,0.075873,-0.049080,-0.073486,-0.062489,-0.033022,-0.026894,0.025211,-0.061934,0.043955,-0.079497,-0.069697,0.052950,0.032700,-0.027623,0.033096,0.026848,0.056534,0.039456,-0.031547,0.037660
soccer,0.087659,0.029007,0.035582,0.008142,-0.033767,0.012876,0.039586,-0.006629,0.001187,0.023257,-0.021101,-0.040725,-0.070640,0.050090,0.038158,-0.048656,0.039810,0.001934,0.034671,0.060569,0.017075,0.085736,-0.050497,-0.014436,-0.044594,-0.011939,0.050371,0.047203,0.004612,-0.007791,0.011378,0.058004,0.042525,-0.079519,0.082394,-0.032396,0.018687,-0.035710,0.024975,0.045227,...,0.088185,0.070491,-0.068756,0.047802,0.051650,0.041846,-0.016278,0.062022,-0.015547,0.076339,0.051044,0.025401,0.001397,-0.049575,-0.005566,-0.049107,-0.017106,0.072293,0.024498,-0.074315,0.008824,0.065300,-0.061786,-0.071968,0.001794,0.034305,-0.025480,-0.014776,-0.064578,0.001628,-0.062911,0.029734,0.053855,0.005754,0.004388,0.036906,0.009014,0.044967,0.066259,0.032534
กีฬา,0.124002,-0.010064,0.021586,-0.026046,0.004620,0.013048,0.054116,-0.029797,-0.015586,-0.052211,0.025364,-0.010773,-0.081498,-0.066780,-0.009214,-0.048165,0.007987,0.013987,0.072266,0.050698,0.068910,0.097580,-0.042512,-0.004998,-0.018245,-0.001656,0.091537,-0.002022,-0.003767,-0.019207,0.023712,0.020826,-0.003649,-0.048395,0.088369,-0.018639,0.069581,0.029565,0.027604,0.059889,...,0.124798,0.031045,0.058436,0.013203,0.003478,-0.028695,-0.037012,0.048890,-0.006056,0.070188,0.026254,-0.016243,0.021774,-0.051376,0.070367,-0.088702,0.000663,0.041626,-0.033591,-0.110613,0.010898,-0.008416,-0.040983,-0.063179,-0.066491,0.038876,0.006141,0.026581,-0.024507,-0.058071,-0.090682,-0.000999,0.015031,-0.031466,0.000742,-0.030618,0.062340,0.014764,-0.032761,0.017188
สัตว์เลี้ยง,0.126926,-0.012773,0.003140,0.024147,-0.015919,0.013569,0.000303,0.005081,-0.068021,-0.023217,-0.033968,-0.046152,-0.073648,-0.012267,0.025458,-0.015771,0.040347,0.011427,0.046440,0.007424,0.084326,0.089551,-0.045606,0.024244,-0.055126,-0.028851,0.051724,0.004162,0.065311,0.011024,-0.023513,-0.036925,-0.043011,-0.040230,0.038276,-0.043044,-0.009691,0.050832,0.026739,0.048669,...,-0.023664,-0.038064,0.058306,0.025154,-0.029802,0.012536,-0.045402,-0.012059,0.037537,0.079541,-0.004009,-0.048719,-0.122716,0.093443,0.045741,-0.091553,-0.017403,0.038938,-0.037284,-0.014414,-0.010770,-0.075816,0.118343,-0.070053,-0.051365,-0.014079,0.027170,0.044875,-0.134388,0.008657,-0.106966,-0.039584,-0.003838,-0.018855,0.038801,0.003765,0.010519,-0.053808,-0.005291,-0.030242
เทคโนโลยี,0.121281,0.067098,0.035793,0.046788,-0.003407,-0.079176,-0.023236,0.014330,0.045633,-0.018322,-0.037618,-0.014100,-0.081966,-0.063503,-0.012012,-0.028219,0.037737,0.008750,-0.018160,0.037616,0.052570,0.068460,0.023603,-0.041780,-0.015976,0.017943,0.003187,-0.053067,0.037429,-0.031724,-0.019879,0.006650,-0.026393,-0.015123,-0.027205,-0.052168,0.050678,-0.069555,-0.008986,0.044261,...,-0.061506,0.037574,0.014158,0.045707,-0.028640,0.067216,-0.031968,0.014527,0.044679,0.062199,-0.039211,0.089034,-0.027605,-0.035559,0.053428,-0.075589,0.052525,-0.062822,0.030026,0.001451,0.056872,-0.067867,-0.024339,-0.055131,-0.028915,0.009248,-0.020097,-

### Categorize to Main Categories by Embed Selected Categories into Vector Space. Then use Dot Product Similarity to get The Closest Category using Argmax Index.

In [20]:
set_category = ['กีฬา', 'เลี้ยงสัตว์', 'ดูดวง', 'รถยนต์', 'เทคโนโลยี','ปลูกต้นไม้','ถ่ายภาพ',
                'ชอบผู้ชาย','ชอบผู้หญิง','นอน','ดนตรี','ขายของ','การ์ตูน','ท่องเที่ยว','ความรู้','ข่าว'
                'ซีรีย์', 'อาหาร', 'ดารา','ออกกำลังกาย','ลงทุน','วิ่งมาราธอน','ซื้อของ','ธรรมะ']

In [21]:
df_category = pd.DataFrame(embed(set_category).numpy(), index = set_category)
other_interest['กลุ่มความสนใจ'] = df_use_oth_interest.dot(df_category.T).idxmax('columns').values
other_interest.head(30)

,index,ความสนใจอื่นๆ,กลุ่มความสนใจ
0,0,กีฬาufc,กีฬา
1,0,soccer,กีฬา
2,1,กีฬา,กีฬา
3,1,สัตว์เลี้ยง,เลี้ยงสัตว์
4,1,เทคโนโลยี,เทคโนโลยี
5,3,โหราศาสตร์,ดูดวง
6,3,ดูดวง,ดูดวง
7,4,สัตว์เลี้ยง,เลี้ยงสัตว์
8,5,ด้านการออมเงินวางแผนการใช้เงิน,ลงทุน
9,6,อุปกรณ์ไอที,เทคโนโลยี


In [22]:
other_interest['กลุ่มความสนใจ'].value_counts()

เลี้ยงสัตว์    12
กีฬา           12
ลงทุน          11
รถยนต์         11
ดนตรี          10
เทคโนโลยี       9
ความรู้         7
ดูดวง           6
อาหาร           5
การ์ตูน         5
ข่าวซีรีย์      4
ปลูกต้นไม้      3
ชอบผู้หญิง      3
นอน             3
ธรรมะ           3
ขายของ          3
ซื้อของ         3
ถ่ายภาพ         2
ดารา            2
ชอบผู้ชาย       2
ท่องเที่ยว      1
วิ่งมาราธอน     1
Name: กลุ่มความสนใจ, dtype: int64

In [23]:
other_interest[other_interest['กลุ่มความสนใจ']=='ธรรมะ']

,index,ความสนใจอื่นๆ,กลุ่มความสนใจ
57,36,ธรรมมะ,ธรรมะ
69,42,การทำสมาธิ,ธรรมะ
111,58,ธรรมะ,ธรรมะ


In [24]:
other_interest.groupby('กลุ่มความสนใจ')[['index']].nunique().sort_values(by='index', ascending=False)

,index
กลุ่มความสนใจ,
ลงทุน,11
เลี้ยงสัตว์,10
เทคโนโลยี,9
รถยนต์,9
กีฬา,9
ดนตรี,8
ความรู้,6
ดูดวง,5
อาหาร,5


In [25]:
other_interest.to_csv('others_interest.csv', index=False)

# Freqeunt Interest Items Set Analysis Using Apriori

In [26]:
df_fiis = other_interest.pivot_table(index='index', columns = 'กลุ่มความสนใจ', values = 'ความสนใจอื่นๆ', aggfunc = 'count').fillna(0)
df_fiis = df_fiis.applymap(lambda x: 1 if x >= 1 else 0)
df_fiis.head()

กลุ่มความสนใจ,การ์ตูน,กีฬา,ขายของ,ข่าวซีรีย์,ความรู้,ชอบผู้ชาย,ชอบผู้หญิง,ซื้อของ,ดนตรี,ดารา,ดูดวง,ถ่ายภาพ,ท่องเที่ยว,ธรรมะ,นอน,ปลูกต้นไม้,รถยนต์,ลงทุน,วิ่งมาราธอน,อาหาร,เทคโนโลยี,เลี้ยงสัตว์
index,,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [27]:
from mlxtend.frequent_patterns import apriori, association_rules 

In [28]:
# Building the model 
frq_items = apriori(df_fiis, min_support = 0.03, use_colnames = True)

In [29]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['support', 'lift'], ascending =[False, False]) 
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(กีฬา),(รถยนต์),0.150000,0.150000,0.050000,0.333333,2.222222,0.027500,1.275000
1,(รถยนต์),(กีฬา),0.150000,0.150000,0.050000,0.333333,2.222222,0.027500,1.275000
4,(ชอบผู้หญิง),(ถ่ายภาพ),0.050000,0.033333,0.033333,0.666667,20.000000,0.031667,2.900000
5,(ถ่ายภาพ),(ชอบผู้หญิง),0.033333,0.050000,0.033333,1.000000,20.000000,0.031667,inf
6,(ดนตรี),(รถยนต์),0.133333,0.150000,0.033333,0.250000,1.666667,0.013333,1.133333
7,(รถยนต์),(ดนตรี),0.150000,0.133333,0.033333,0.222222,1.666667,0.013333,1.114286
8,(เทคโนโลยี),(ดนตรี),0.150000,0.133333,0.033333,0.222222,1.666667,0.013333,1.114286
9,(ดนตรี),(เทคโนโลยี),0.133333,0.150000,0.033333,0.250000,1.666667,0.013333,1.133333
10,(ดนตรี),(เลี้ยงสัตว์),0.133333,0.166667,0.033333,0.250000,1.500000,0.011111,1.111111
11,(เลี้ยงสัตว์),(ดนตรี),0.166667,0.133333,0.033333,0.200000,1.500000,0.011111,1.083333


In [30]:
rules.to_csv('cb_rules.csv', index=False)